We begin by importing the texts extracted using pdf miner.

In [34]:
#We import the necessary Packages
import pandas as pd
import re
import numpy as np
import datetime
import os.path
import random

DATA_DIR = os.path.abspath('/Users/attiliocastano/Documents/GitHub/amethyst-erdos-may-2021/data')

In [35]:
DATA_DIR

'/Users/attiliocastano/Documents/GitHub/amethyst-erdos-may-2021/data'

In [40]:
PDFMINER_TEXT_DIR = os.path.join(DATA_DIR,'pdfminer_texts')
pdfminer_text_filenames = os.listdir(PDFMINER_TEXT_DIR)

In [50]:
random_text_names = random.choices(pdfminer_text_filenames, k = 100)

In [52]:
pdfminer_texts = []
for filename in random_text_names:
    with open(os.path.join(PDFMINER_TEXT_DIR,filename),'r') as f:
        pdfminer_texts.append(f.read())

In [54]:
Files_pdfminer = pd.DataFrame({'filename': random_text_names,
                              'extracted_text':pdfminer_texts})

In [55]:
Files_pdfminer.head()

,filename,extracted_text
0,1993_july_1_803961468286495268_conformed-copy-...,CONFORMED\n\n ...
1,2004_november_1_176131468224998059_loan-agreem...,CONFORMED COPY \n\n \nLOAN NUMBER 4746-CHA \n\...
2,2000_august_10_472421468223191843_conformed-co...,CONFORMED COPY\n\nLOAN NUMBER 4570 CHA\n\nLoan...
3,2006_september_26_405771468112484727_loan-agre...,CONFORMED COPY \n \n\nLOAN NUMBER 4836-UA \n\n...
4,1999_june_17_975471468094790343_conformed-copy...,CONFORMED COPY\n\nLOAN NUMBER 4491 RO\n\n(Priv...


We attempt to use TensorFlow to extract features from the texts

In [63]:
import tensorflow as tf

In [66]:
import collections
import pathlib

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import utils
from tensorflow.keras.layers import TextVectorization

import tensorflow_datasets as tfds
import tensorflow_text as tf_text

In [67]:
Extracted_DF=pd.read_csv(os.path.join(DATA_DIR,'Extracted_Attributes_LoanUSD_Country.csv'))

In [69]:
Extracted_DF.head()

,filename,Project_name,Project_desc,Date,Amount_loan,Currency_name,Year,Value,Amount_in_usd,Country_Code,Country,Region,Income_level
0,1990_april_24_587321468019152780_conformed-cop...,forestry sector project,Description of the Project The objectives of t...,1990-04-24,80000000.0,USD,1990,1.0,80000000.0,CIV,Cote d'Ivoire,Sub-Saharan Africa,Lower middle income
1,1990_april_24_668811468165272290_conformed-cop...,water supply project,NaN,1990-04-24,2500000.0,USD,1990,1.0,2500000.0,LCA,St. Lucia,Latin America & Caribbean,Upper middle income
2,1990_april_25_904191468298750561_conformed-cop...,environment management project,Description of the Project The objectives of t...,1990-04-25,18000000.0,USD,1990,1.0,18000000.0,POL,Poland,Europe & Central Asia,High income
3,1990_april_30_410811468040573756_conformed-cop...,rural electrification project,Description of the Project The objective of th...,1990-04-30,329000000.0,USD,1990,1.0,329000000.0,IDN,Indonesia,East Asia & Pacific,Upper middle income
4,1990_april_30_725911468042268845_conformed-cop...,third telecommunications project,Description of the Project The objectives of t...,1990-04-30,350000000.0,USD,1990,1.0,350000000.0,IDN,Indonesia,East Asia & Pacific,Upper middle income


In [81]:
T = Extracted_DF.loc[4, "Project_desc"]
print(T)
type(T)

Description of the Project The objectives of the Project are to assist the Borrower to improve overall management of the telecommunications sector develop PERUMTELs institutional capability and accelerate implementation of telecommunications network investments The Project consists of the following parts subject to such modifications thereof as the Borrower and the Bank may agree upon from time to time to achieve such objectives


str

In [124]:
VOCAB_SIZE = 100

binary_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='binary')

In [125]:
MAX_SEQUENCE_LENGTH = 250

int_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

In [193]:
text_dataset = tf.convert_to_tensor(T.split())
text_dataset.shape

TensorShape([62])

In [179]:
binary_vectorize_layer.adapt(text_dataset)

In [197]:
# Create the model that uses the vectorize text layer
model = tf.keras.models.Sequential()

# Start by creating an explicit input layer. It needs to have a shape of
# (1,) (because we need to guarantee that there is exactly one string
# input per batch), and the dtype needs to be 'string'.
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))

# The first layer in our model is the vectorization layer. After this
# layer, we have a tensor of shape (batch_size, max_len) containing vocab
# indices.
model.add(binary_vectorize_layer)



In [198]:
print(model.predict(text_dataset))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [189]:
model.predict(text_dataset).shape

(62, 39)

In [200]:
int_vectorize_layer.get_vocabulary()

['',
 '[UNK]',
 'the',
 'to',
 'of',
 'project',
 'time',
 'telecommunications',
 'such',
 'objectives',
 'borrower',
 'and',
 'upon',
 'thereof',
 'subject',
 'sector',
 'perumtels',
 'parts',
 'overall',
 'network',
 'modifications',
 'may',
 'management',
 'investments',
 'institutional',
 'improve',
 'implementation',
 'from',
 'following',
 'develop',
 'description',
 'consists',
 'capability',
 'bank',
 'assist',
 'as',
 'are',
 'agree',
 'achieve',
 'accelerate']

In [191]:
embedding_layer = tf.keras.layers.Embedding(61, 4)

In [185]:
result = embedding_layer(model.predict(text_dataset))
result.numpy()


array([[[-0.00074903,  0.02337625,  0.00013349,  0.02313804],
        [-0.00074903,  0.02337625,  0.00013349,  0.02313804],
        [-0.00074903,  0.02337625,  0.00013349,  0.02313804],
        ...,
        [-0.00074903,  0.02337625,  0.00013349,  0.02313804],
        [-0.00074903,  0.02337625,  0.00013349,  0.02313804],
        [-0.00074903,  0.02337625,  0.00013349,  0.02313804]],

       [[-0.00074903,  0.02337625,  0.00013349,  0.02313804],
        [-0.00074903,  0.02337625,  0.00013349,  0.02313804],
        [-0.00074903,  0.02337625,  0.00013349,  0.02313804],
        ...,
        [-0.00074903,  0.02337625,  0.00013349,  0.02313804],
        [-0.00074903,  0.02337625,  0.00013349,  0.02313804],
        [-0.00074903,  0.02337625,  0.00013349,  0.02313804]],

       [[-0.00074903,  0.02337625,  0.00013349,  0.02313804],
        [-0.00061349,  0.0449929 , -0.04672283,  0.00196093],
        [-0.00074903,  0.02337625,  0.00013349,  0.02313804],
        ...,
        [-0.00074903,  0.02

In [186]:
result.shape

TensorShape([62, 39, 4])